In [2]:
import os
os.getcwd()

'/Users/zhuonijie/models/research/delf/delf/python/examples'

In [3]:
%cd "/Users/zhuonijie/models/research"

/Users/zhuonijie/models/research


In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial import cKDTree
from skimage.feature import plot_matches
from skimage.measure import ransac
from skimage.transform import AffineTransform
import tensorflow as tf

from tensorflow.python.platform import app
from delf import feature_io
import glob
import re

cmd_args = None

_DISTANCE_THRESHOLD = 0.8


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



# 9. DELF features

After extracting DELF features and visualizing DELF examples in command line

In [5]:
images = []
# list_images contains the image paths for original and augmentation images
with open("delf/delf/python/examples/list_images.txt") as mytxt:
    for line in mytxt:
        images.append(line)

In [8]:
features = glob.glob("delf/delf/python/examples/data/MantaRay_features/*")

In [9]:
features_1 = features[0]
features_2 = features[176]
print(features_1)
print(features_2)

delf/delf/python/examples/data/MantaRay_features/AUG_Gamble Sep 15th 2007_EH.JPG_0_3776.delf
delf/delf/python/examples/data/MantaRay_features/AUG_Arnulv_EH.jpg_0_2709.delf


In [10]:
# Read features.
locations_1, _, descriptors_1, _, _ = feature_io.ReadFromFile(features_1)
num_features_1 = locations_1.shape[0]
tf.logging.info("Loaded image 1's %d features" % num_features_1)

locations_2, _, descriptors_2, _, _ = feature_io.ReadFromFile(features_2)
num_features_2 = locations_2.shape[0]
tf.logging.info("Loaded image 2's %d features" % num_features_2)

Instructions for updating:
Use tf.gfile.GFile.
INFO:tensorflow:Loaded image 1's 246 features
INFO:tensorflow:Loaded image 2's 25 features


In [16]:
labels = []
for i in range(len(features)):
    a = features[i].split('/')[-1:][0][:-5]
    a = a.replace('.', '_')
    a = re.split('_',a)
    ind = a.index('EH')-1
    labels.append(a[ind])

In [11]:
# Find nearest-neighbor matches using a KD tree.
d1_tree = cKDTree(descriptors_1)
_, indices = d1_tree.query(descriptors_2, distance_upper_bound=_DISTANCE_THRESHOLD)

In [12]:
# Select feature locations for putative matches.
locations_2_to_use = np.array([
      locations_2[i,]
      for i in range(num_features_2)
      if indices[i] != num_features_1
  ])
locations_1_to_use = np.array([
      locations_1[indices[i],]
      for i in range(num_features_2)
      if indices[i] != num_features_1
  ])

# 10. DELF feature matching

In [189]:
new_scores = []
for i in range(719):
    print(i)
    features_1 = features[i]
    locations_1, _, descriptors_1, _, _ = feature_io.ReadFromFile(features_1)
    num_features_1 = locations_1.shape[0]
    similarities = []
    for j in range(i+1,720):
        features_2 = features[j]
        locations_2, _, descriptors_2, _, _ = feature_io.ReadFromFile(features_2)
        num_features_2 = locations_2.shape[0]
        
        d1_tree = cKDTree(descriptors_1)
        _, indices = d1_tree.query(descriptors_2, distance_upper_bound=_DISTANCE_THRESHOLD)

        locations_2_to_use = np.array([
        locations_2[i,]
        for i in range(num_features_2)
          if indices[i] != num_features_1
      ])
        locations_1_to_use = np.array([
          locations_1[indices[i],]
        for i in range(num_features_2)
          if indices[i] != num_features_1
      ])
        
        if len(locations_1_to_use)==0 | len(locations_2_to_use)==0:
            similarity = 0
            similarities.append(similarity)
            continue
        
        _, inliers = ransac(
      (locations_1_to_use, locations_2_to_use),
      AffineTransform,
      min_samples=3,
      residual_threshold=20,
      max_trials=1000)
        
        if inliers is None:
            similarity = 0
            similarities.append(similarity)
            continue
        
        # Define how similar they are
        number_keypoints = 0
        if num_features_1 <= num_features_2:
            number_keypoints = num_features_1
        else:
            number_keypoints = num_features_2

        similarity = sum(inliers)/number_keypoints
        similarities.append(similarity)
    new_scores.append(similarities)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142


KeyboardInterrupt: 

In [190]:
new_score_list = []
for i in range(len(new_scores)):
    new_score_tmp = new_scores[i]
    new_score_tmp.insert(0,1)
    for j in reversed(range(i)):
        new_score_tmp.insert(0,new_scores[j][i-j])
    new_score_list.append(new_score_tmp)

In [191]:
total_1 = 0
total_5 = 0
total_10 = 0
total_25 = 0
total_50 = 0
total_75 = 0
total_100 = 0
true_1 = 0
true_5 = 0
true_10 = 0
true_25 = 0
true_50 = 0
true_75 = 0
true_100 = 0

for i in range(len(new_score_list)):
        true_label = labels[i]
        top_1_idx = np.argsort(new_score_list[i])[-1:]
        top_5_idx = np.argsort(new_score_list[i])[-5:]
        top_10_idx = np.argsort(new_score_list[i])[-10:]
        top_25_idx = np.argsort(new_score_list[i])[-25:]
        top_50_idx = np.argsort(new_score_list[i])[-50:]
        top_75_idx = np.argsort(new_score_list[i])[-75:]
        top_100_idx = np.argsort(new_score_list[i])[-100:]
        total_1 += 1
        total_5 += 1
        total_10 += 1
        total_25 += 1
        total_50 += 1
        total_75 += 1
        total_100 += 1
        for index in top_1_idx:
            #if index < len(aug_image_labels):
                if labels[index] == true_label:
                    #print(i)
                    true_1 += 1
                    break
        for index in top_5_idx:
            #if index < len(aug_image_labels):
                if labels[index] == true_label:
                    true_5 += 1
                    break
        for index in top_10_idx:
            #if index < len(aug_image_labels):
                if labels[index] == true_label:
                    true_10 += 1
                    break
        for index in top_25_idx:
            #if index < len(aug_image_labels):
                if labels[index] == true_label:
                    true_25 += 1
                    break
        for index in top_50_idx:
            #if index < len(aug_image_labels):
                if labels[index] == true_label:
                    true_50 += 1
                    break
        for index in top_75_idx:
            #if index < len(aug_image_labels):
                if labels[index] == true_label:
                    true_75 += 1
                    break
        for index in top_100_idx:
            #if index < len(aug_image_labels):
                if labels[index] == true_label:
                    true_100 += 1
                    break

In [192]:
print(true_1/total_1)
print(true_5/total_5)
print(true_10/total_10)
print(true_25/total_25)
print(true_50/total_50)
print(true_75/total_75)
print(true_100/total_100)

0.5422535211267606
1.0
1.0
1.0
1.0
1.0
1.0


In [27]:
with open('delf_testset.txt', 'w') as f:
    for item in features[:142]:
        f.write("%s\n" % item)